# This is currently a work in progress.  Feel free to add to it as it progresses along!!!

## This notebook is a gradual work in progress to make this data set more complete and easier for people to import and use for their own data analysis with minimal data cleaning.

In [1]:
import pandas as pd
import os

## Read in the raw data files and combine into one data frame

In [2]:
file_list = os.listdir('raw_data/')

In [75]:
d = pd.concat([pd.read_csv('raw_data/' + f) for f in file_list], keys=file_list, names = ['FileName','RowNumber'])

In [118]:
df = d.reset_index().drop('RowNumber', axis = 1)

## Load in neighborhood lookup table and join it with the data frame

In [119]:
# Load in neighborhood lookup table
nbhds = pd.read_csv('neighborhood_lookup.csv')

In [153]:
df2 = pd.merge(df, nbhds, left_on = 'Neighborhood', right_on = 'Neighborhood Number', how = 'left')
df2.drop('Neighborhood Number', axis = 1, inplace = True) # Drop duplicate nbhd number column

## Fixing column names to something more consistent

In [155]:
col_names = list(df2.columns)
df2.columns = [name.replace(' ', '') for name in col_names]

In [156]:
df2.columns

Index([u'FileName', u'AdministrativeAdjustmentIndicator', u'CADAddress', u'CADStreet', u'CodedMonth', u'Complaint', u'Count', u'Crime', u'DateOccur', u'DateOccured', u'Description', u'District', u'FlagAdministrative', u'FlagCleanup', u'FlagCrime', u'FlagUnfounded', u'ILEADSAddress', u'ILEADSStreet', u'LocationComment', u'LocationName', u'MonthReportedtoMSHP', u'Neighborhood', u'NewCrimeIndicator', u'UnfoundedCrimeIndicator', u'XCoord', u'YCoord', u'NeighborhoodName', u'NeighborhoodPrimaryDistrict', u'NeighborhoodAddlDistrict'], dtype='object')

## Working to fill in missing data

Some fields appear to just be flags for certain things (revisions, what was revised, etc.).  Replacing all the ' ' and NaN (missing data) with 'N', leaving any 'Y' in place

In [167]:
df2['AdministrativeAdjustmentIndicator'] = df2['AdministrativeAdjustmentIndicator'].replace(' ', 'N')
df2['AdministrativeAdjustmentIndicator'] = df2['AdministrativeAdjustmentIndicator'].replace(pd.np.NaN, 'N')
df2['NewCrimeIndicator'] = df2['NewCrimeIndicator'].replace(' ', 'N')
df2['NewCrimeIndicator'] = df2['NewCrimeIndicator'].replace(pd.np.NaN, 'N')
df2['FlagAdministrative'] = df2['FlagAdministrative'].replace(' ', 'N')
df2['FlagAdministrative'] = df2['FlagAdministrative'].replace(pd.np.NaN, 'N')
df2['FlagCleanup'] = df2['FlagCleanup'].replace(' ', 'N')
df2['FlagCleanup'] = df2['FlagCleanup'].replace(pd.np.NaN, 'N')
df2['FlagCrime'] = df2['FlagCrime'].replace(' ', 'N')
df2['FlagCrime'] = df2['FlagCrime'].replace(pd.np.NaN, 'N')
df2['FlagUnfounded'] = df2['FlagUnfounded'].replace(' ', 'N')
df2['FlagUnfounded'] = df2['FlagUnfounded'].replace(pd.np.NaN, 'N')
df2['UnfoundedCrimeIndicator'] = df2['UnfoundedCrimeIndicator'].replace(' ', 'N')
df2['UnfoundedCrimeIndicator'] = df2['UnfoundedCrimeIndicator'].replace(pd.np.NaN, 'N')

# Filling out some other fields with NaN's appropriately
df2['LocationName'] = df2['LocationName'].replace(' ', pd.np.NaN)
df2['LocationComment'] = df2['LocationComment'].replace(' ', pd.np.NaN)

## Here we're trying to align the crime codes in the Crime field to the Uniform Crime Reporting codes.
First we need to pad out the Crime column to be six digits, since the leading zero was trimmed in the .csv files

*Need to figure out the numbering system for these crimes.  See UCR, NIBRS, NCIC crime codes and try to make sense of it.*

In [158]:
df2['Crime'] = df2['Crime'].astype(str)
df2['Crime'] = df2['Crime'].apply(lambda x: x.zfill(6))

## Here we're merging the DateOccur and DateOccured columns, which are disjoint, but give the same information.
Combine DateOccur and DateOccured columns (no overlap confirmed, they must've just renamed it at some point)

In [179]:
df2[['DateOccur','DateOccured']]

,DateOccur,DateOccured
0,NaN,01/01/2000 20:58
1,NaN,01/01/2003 00:01
2,NaN,01/01/2004 00:01
3,NaN,01/01/2006 10:00
4,NaN,01/01/2007 16:36
5,NaN,01/01/2008 00:30
6,NaN,01/01/2008 08:00
7,NaN,01/01/2008 13:45
8,NaN,01/03/2008 09:00
9,NaN,01/05/2008 21:37


### The below code merges the two columns without overlapping anything

In [182]:
df2['DateOccured'] = pd.concat([df2['DateOccur'].dropna(), df['DateOccured'].dropna()]).reindex_like(df2)

In [186]:
df2

,FileName,AdministrativeAdjustmentIndicator,CADAddress,CADStreet,CodedMonth,Complaint,Count,Crime,DateOccur,DateOccured,...,LocationName,MonthReportedtoMSHP,Neighborhood,NewCrimeIndicator,UnfoundedCrimeIndicator,XCoord,YCoord,NeighborhoodName,NeighborhoodPrimaryDistrict,NeighborhoodAddlDistrict
0,april2008.csv,N,4900,MAPLE,NaN,NaN,1,177000,NaN,01/01/2000 20:58,...,NaN,2008-04,53,Y,N,888708.3,1028833.0,Fountain Park,8,NaN
1,april2008.csv,N,4357,CALIFORNIA,NaN,NaN,1,067501,NaN,01/01/2003 00:01,...,NaN,2008-04,17,Y,N,896256.1,998745.9,Mount Pleasant,1,NaN
2,april2008.csv,N,4313,MARGARETTA,NaN,NaN,1,115400,NaN,01/01/2004 00:01,...,NaN,2008-04,68,Y,N,896282.4,1033555.0,O'Fallon,5,6
3,april2008.csv,N,3939,OHIO,NaN,NaN,1,115400,NaN,01/01/2006 10:00,...,NaN,2008-04,16,Y,N,897347.3,1001419.0,Dutchtown,1,NaN
4,april2008.csv,N,2612,ANNIE MALONE DR,NaN,NaN,1,091113,NaN,01/01/2007 16:36,...,NaN,2008-04,57,Y,N,895420.8,1029119.0,The Ville,8,NaN
5,april2008.csv,N,5500,DELMAR,NaN,NaN,1,021000,NaN,01/01/2008 00:30,...,NaN,2008-04,49,Y,N,883561.5,1027076.0,Visitation Park,7,NaN
6,april2008.csv,N,4465,MANCHESTER,NaN,NaN,1,067601,NaN,01/01/2008 08:00,...,RENARD PAPER COMPANY,2008-04,39,Y,N,888440.3,1017117.0,Forest Park SE,9,NaN
7,april2008.csv,N,3628,MORGANFORD,NaN,NaN,1,044026,NaN,01/01/2008 13:45,...,NaN,2008-04,15,Y,N,888024.2,1004480.0,Tower Grove South,3,NaN
8,april2008.csv,N,1034,S KINGSHIGHWAY,NaN,NaN,1,264100,NaN,01/03/2008 09:00,...,NaN,2008-04,0,Y,N,0.0,0.0,Unknown,NaN,NaN
9,april2008.csv,N,1115,HAMPTON,NaN,NaN,1,117000,NaN,01/05/2008 21:37,...,NaN,2008-04,42,Y,N,880910.0,1018021.0,Clayton-Tamm,2,NaN


## Write the file out to tab delimited file

In [95]:
df2.to_csv('clean_data/stl_crime_data.txt', sep='\t')